# Effect of active conductances on LFP from single synaptic input to pyramidal cell model from Hay et al. (2011)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack as ff
import LFPy
import neuron
import brainsignals.neural_simulations as ns
from brainsignals.plotting_convention import mark_subplots, simplify_axes

ns.load_mechs_from_folder(ns.cell_models_folder)

np.random.seed(12345)

tstop_syn = 50
tstop_wn = 6000
dt = 2**-5
num_tsteps_wn = int(tstop_wn / dt)
tvec_syn = np.arange(int(tstop_syn / dt + 1)) * dt

tvec_wn = np.arange(int(tstop_wn / dt + 1)) * dt
tvec_wn_cutoff_idx = np.argmin(np.abs(tvec_wn - tstop_wn + 1000)) + 1

sample_freq = ff.fftfreq(len(tvec_wn[tvec_wn_cutoff_idx:]), d=dt / 1000)
pidxs = np.where(sample_freq >= 0)

stim_freqs = sample_freq[pidxs]

min_stim_freq = 1
max_stim_freq = 1000
min_stim_freq_idx = np.argmin(np.abs(stim_freqs - min_stim_freq))
max_stim_freq_idx = np.argmin(np.abs(stim_freqs - max_stim_freq))


In [ ]:
def make_WN_input(cell, freqs):
    """ White Noise input ala Linden 2010 is made """
    tot_ntsteps = round((cell.tstop - cell.tstart) / cell.dt + 1)
    I = np.zeros(tot_ntsteps)
    tvec = np.arange(tot_ntsteps) * cell.dt
    for freq in freqs:
        I += np.sin(2 * np.pi * freq * tvec/1000. + 2*np.pi*np.random.random())
    return I


def make_white_noise_stimuli(cell, input_idx, freqs, weight=None):

    input_scaling = 0.0005
    np.random.seed(1234)
    input_array = input_scaling * (make_WN_input(cell, freqs))

    noise_vec = (neuron.h.Vector(input_array) if weight is None
                 else neuron.h.Vector(input_array * weight))

    i = 0
    syn = None
    for sec in cell.allseclist:
        for seg in sec:
            if i == input_idx:
                print("Input inserted in ", sec.name())
                syn = neuron.h.ISyn(seg.x, sec=sec)
            i += 1
    if syn is None:
        raise RuntimeError("Wrong stimuli index")
    syn.dur = 1E9
    syn.delay = 0
    noise_vec.play(syn._ref_amp, cell.dt)
    return cell, syn, noise_vec


def insert_synaptic_input(cell, synidx):

    synapse_parameters = dict(
                          idx = synidx,
                          e = 0, # reversal potential
                          weight = 0.001, # synapse weight
                          record_current = False, # record synapse current
                          syntype = 'Exp2Syn',
                          tau1 = 0.5, #Time constant, rise
                          tau2 = 2.0, #Time constant, decay
                          )
    synapse = LFPy.Synapse(cell, **synapse_parameters)
    synapse.set_spike_times(np.array([3.]))
    return synapse, cell

elec_x = np.array([100,  120])
elec_z = np.array([0, 1000])
elec_y = np.array([0, 0])

elec_params = dict(
            sigma = 0.3,      # extracellular conductivity
            x = elec_x,
            y = elec_y,
            z = elec_z,
        )


In [ ]:
cell = ns.return_hay_cell(tstop=tstop_syn, dt=dt, make_passive=False)
ns.point_axon_down(cell)

synidxs = [cell.get_closest_idx(x=0, y=0, z=1000),
           cell.get_closest_idx(x=0, y=0, z=0)]

sim_cases = ["passive", "active", "passive + Ih", ]



input_types = ["synapse", "white noise"]

sim_clrs = {"active": 'r',
           "passive + Ih": 'gray',
           "passive": 'k'}
sim_lss = {"active": '-',
           "passive + Ih": ':',
           "passive": '-'}

remove_lists = {"active": [],
                "passive + Ih": ["Nap_Et2", "NaTa_t", "NaTs2_t", "SKv3_1",
                                 "SK_E2", "K_Tst", "K_Pst", #"Ih", 
                                 "Im", "CaDynamics_E2", "Ca_LVAst", "Ca", "Ca_HVA"],
                "passive": ["Nap_Et2", "NaTa_t", "NaTs2_t", "SKv3_1",
                            "SK_E2", "K_Tst", "K_Pst", "Ih", 
                            "Im", "CaDynamics_E2", "Ca_LVAst", "Ca", "Ca_HVA"],
               }

elec = LFPy.RecExtElectrode(cell, **elec_params)
M_elec = elec.get_transformation_matrix()
del cell

v_es = {}

for s_idx, sim_case in enumerate(sim_cases):
    v_es[sim_case] = {}

    for input_type in input_types:
        tstop_ = tstop_syn if input_type == "synapse" else tstop_wn
        v_es[sim_case][input_type] = {}
        
        for sim_idx in range(len(synidxs)):
            cell = ns.return_hay_cell(tstop=tstop_, dt=dt, make_passive=False)
            if input_type == "synapse":
                syn, cell = insert_synaptic_input(cell, synidxs[sim_idx])
            elif input_type == "white noise":
                cell, syn, nv = make_white_noise_stimuli(cell, synidxs[sim_idx], 
                                                         stim_freqs[min_stim_freq_idx:max_stim_freq_idx])

            ns.point_axon_down(cell)
            cell = ns.remove_active_mechanisms(remove_lists[sim_case], cell)

            cell.simulate(rec_imem=True, rec_vmem=True)
            print(sim_case, input_type, sim_idx, "Max dV: ", np.max(np.abs(cell.vmem - cell.vmem[:, 0, None])))
            
            v_e = M_elec @ cell.imem * 1000
            
            if input_type == "synapse":
                v_e -= v_e[:, 0, None]
                v_es[sim_case][input_type][sim_idx] = v_e.copy()
            elif input_type == "white noise":
                v_e = v_e[:, tvec_wn_cutoff_idx:]
                cell.tvec = cell.tvec[tvec_wn_cutoff_idx:] - cell.tvec[tvec_wn_cutoff_idx]
                cell.vmem = cell.vmem[:, tvec_wn_cutoff_idx:]
                cell.imem = cell.imem[:, tvec_wn_cutoff_idx:]
                freqs, psd = ns.return_freq_and_psd(cell.tvec, v_e)
                v_es[sim_case][input_type][sim_idx] = psd
            del cell

In [ ]:
xmin = -145
xmax = 220
zmin = -200
zmax = 1200

fig = plt.figure(figsize=[6, 3])
fig.subplots_adjust(bottom=0.17, top=0.9, left=0.01, right=0.99, wspace=0.8, hspace=0.6)

ax1 = fig.add_axes([0, 0, 0.15, 1], frameon=False, aspect=1,
                        xticks=[], yticks=[], xlim=[xmin, xmax],
                        ylim=[zmin, zmax])

ax_ai_ae_s = fig.add_subplot(252, xlabel="time (ms)", ylabel=r"$V_{\rm e}$ (µV)")
ax_ai_se_s = fig.add_subplot(257, xlabel="time (ms)", ylabel=r"$V_{\rm e}$ (µV)")
ax_si_ae_s = fig.add_subplot(253, xlabel="time (ms)", ylabel=r"$V_{\rm e}$ (µV)")
ax_si_se_s = fig.add_subplot(258, xlabel="time (ms)", ylabel=r"$V_{\rm e}$ (µV)")
ax_ai_ae_wn = fig.add_subplot(254, xlabel="frequency (Hz)", ylabel=r"$V_{\rm e}$ PSD (µV²/Hz)", 
                              xlim=[1, 300], ylim=[1e-6, 1e-2])
ax_ai_se_wn = fig.add_subplot(259, xlabel="frequency (Hz)", ylabel=r"$V_{\rm e}$ PSD (µV²/Hz)",
                             xlim=[1, 300], ylim=[1e-6, 1e-2])
ax_si_ae_wn = fig.add_subplot(255, xlabel="frequency (Hz)",  ylabel=r"$V_{\rm e}$ PSD (µV²/Hz)", 
                              xlim=[1, 300], ylim=[1e-6, 1e-2])
ax_si_se_wn = fig.add_subplot(2,5,10, xlabel="frequency (Hz)", ylabel=r"$V_{\rm e}$ PSD (µV²/Hz)", 
                             xlim=[1, 300], ylim=[1e-6, 1e-2])

ax_ai_ae_s.set_title("apical input", color='c')
ax_si_ae_s.set_title("soma input", color='g')

ax_ai_ae_wn.set_title("apical input\nwhite noise", color='c')
ax_si_ae_wn.set_title("soma input\nwhite noise", color='g')

basal_elec_idx = 0
apical_elec_idx = 1

soma_input_idx = 1
apical_input_idx = 0

lines = []
line_names = []
for s_idx, sim_case in enumerate(sim_cases):
    l, = ax_ai_ae_s.plot(tvec_syn, v_es[sim_case]["synapse"][apical_input_idx][apical_elec_idx], 
                    c=sim_clrs[sim_case], ls=sim_lss[sim_case])
    ax_ai_se_s.plot(tvec_syn, v_es[sim_case]["synapse"][apical_input_idx][basal_elec_idx], 
                    c=sim_clrs[sim_case], ls=sim_lss[sim_case])
    ax_si_ae_s.plot(tvec_syn, v_es[sim_case]["synapse"][soma_input_idx][apical_elec_idx], 
                    c=sim_clrs[sim_case], ls=sim_lss[sim_case])
    ax_si_se_s.plot(tvec_syn, v_es[sim_case]["synapse"][soma_input_idx][basal_elec_idx], 
                    c=sim_clrs[sim_case], ls=sim_lss[sim_case])
    
    ax_ai_ae_wn.loglog(freqs, v_es[sim_case]["white noise"][apical_input_idx][apical_elec_idx], 
                    c=sim_clrs[sim_case], ls=sim_lss[sim_case])
    ax_ai_se_wn.loglog(freqs, v_es[sim_case]["white noise"][apical_input_idx][basal_elec_idx], 
                    c=sim_clrs[sim_case], ls=sim_lss[sim_case])
    ax_si_ae_wn.loglog(freqs, v_es[sim_case]["white noise"][soma_input_idx][apical_elec_idx], 
                    c=sim_clrs[sim_case], ls=sim_lss[sim_case])
    ax_si_se_wn.loglog(freqs, v_es[sim_case]["white noise"][soma_input_idx][basal_elec_idx], 
                    c=sim_clrs[sim_case], ls=sim_lss[sim_case])
    lines.append(l)
    if sim_case == "passive + Ih":
        
        line_names.append(r"passive + $I_{\rm h}$")
    else:
        line_names.append(sim_case)

cell = ns.return_hay_cell(tstop=tstop_syn, dt=dt, make_passive=False)
ns.point_axon_down(cell)
ax1.plot(cell.x.T, cell.z.T, c='0.9', lw=1)
        
ax1.plot(cell.x[synidxs[0]].mean(),
         cell.z[synidxs[0]].mean(), c='c',
                       marker='o', ls='none')
ax1.plot(cell.x[synidxs[1]].mean(),
         cell.z[synidxs[1]].mean(), c='g',
                       marker='o', ls='none')

ax1.plot(elec_x, elec_z, 'o', c='k', ms=3)

ax1.arrow(elec_x[0], elec_z[0], 400, 50, head_width=0, color='k', clip_on=False, lw=0.5)
ax1.arrow(elec_x[1], elec_z[1], 400, -20, head_width=0, color='k', clip_on=False, lw=0.5)


fig.legend(lines, line_names, frameon=False, ncol=3, loc=(0.3, 0.001))

simplify_axes(fig.axes)
mark_subplots(ax1, xpos=0.05, ypos=0.96)
mark_subplots([ax_ai_ae_s, ax_si_ae_s, ax_ai_ae_wn, ax_si_ae_wn], ["B", "C", "D", "E"], ypos=1.22)

fig.savefig("hay_active_conductances_alt.pdf")